In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("../../wy_private/walmart-triptype-classification/dataset/train.csv")
original_train = train.copy()
test = pd.read_csv("../../wy_private/walmart-triptype-classification/dataset/test.csv")
original_test = test.copy()

# Train

In [107]:
train = original_train.copy()

In [108]:
# 반품 발생은 0으로 표기하는 컬럼 추가
# train["ScanCount"] = train["ScanCount"]
# train.loc[train.ScanCount < 0, "ScanCount"] = 0

# Fineline 없는 것은 9999로 명시
train["FinelineNumber"].fillna(value=9999, inplace=True)

# Upc 없는 것은 -1로 명시
train["Upc"].fillna(value=-1, inplace=True)

In [109]:
wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])

In [110]:
train = train[train['VisitNumber'] != 2154].reset_index(drop=True)

In [111]:
# VisitNumber, DepartmentDescription으로 groupby. ScanCount sum(총 판매량) 
df = train.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].sum()

df1 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].min()
df1.rename(columns={"ScanCount": "Min"}, inplace=True)

df2 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].max()
df2.rename(columns={'ScanCount': "Max"}, inplace=True)

df3 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].mean()
df3.rename(columns={"ScanCount": "Mean"}, inplace=True)

train = train.merge(df1, how='left', on=["VisitNumber"], copy=True)
train = train.merge(df2, how='left', on=["VisitNumber"], copy=True)
train = train.merge(df3, how='left', on=["VisitNumber"], copy=True)  

train["Range"] = train["Max"] - train["Min"]

In [112]:
df = train[train["DepartmentDescription"].isnull()]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Depts"}, inplace=True)
train = train.merge(df, how='left', on=["VisitNumber"], copy=True) 
train["Null_Depts"].fillna(value=1, inplace=True)
train.loc[train.Null_Depts > 0, "Null_Depts"] = 0

In [113]:
# df = train[train["ScanCount"] < 0]
# df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].ScanCount()
# df.rename(columns={"ScanCount": "ReturnScanCount"}, inplace=True) 
# train = train.merge(df, how='left', on=["VisitNumber"], copy=True)
# train["ReturnScanCount"].fillna(value=1, inplace=True)
# train.loc[train.ReturnScanCount > 0, "ReturnScanCount"] = 0

In [114]:
# Indicator for missing FinelineNumber

df = train[train["FinelineNumber"]==9999]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Fineline"}, inplace=True)
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)
train["Null_Fineline"].fillna(value=1, inplace=True)
train.loc[train.Null_Fineline > 0, "Null_Fineline"] = 0

In [115]:
# 각 VisitNumber 당 총 구매한 FinelineNumber의 수

df = train.groupby(["VisitNumber",  "FinelineNumber"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Fineline"}, inplace=True) 
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)

In [116]:
# 각 소비자 당 총 구매한 UPC의 수

df = train.groupby(["VisitNumber",  "Upc"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Upc"}, inplace=True) 
train = train.merge(df, how='left', on=["VisitNumber"], copy=True) 

In [117]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc
0,999,5,4,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1,1
1,30,7,4,6.053882e+10,1,SHOES,8931.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2
2,30,7,4,7.410811e+09,1,PERSONAL CARE,4504.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2
3,26,8,4,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21
4,26,8,4,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21


In [118]:
# 각 소비자 당 총 구매한 DepartmentDescription의 수

df = train.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].count()

In [119]:
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()

In [120]:
df.rename(columns={"ScanCount": "N_Depts"}, inplace=True) 

In [121]:
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)

In [122]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,N_Depts
0,999,5,4,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1,1,1.0
1,30,7,4,6.053882e+10,1,SHOES,8931.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2,2.0
2,30,7,4,7.410811e+09,1,PERSONAL CARE,4504.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2,2.0
3,26,8,4,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21,6.0
4,26,8,4,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21,6.0


In [123]:
df = train.groupby(["VisitNumber"], as_index=False)["ScanCount"].sum()
df.rename(columns={"ScanCount": "Sum"}, inplace=True)
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)

In [124]:
train["Ratio_F_D"] = train["N_Fineline"] / train["N_Depts"]
train["Ratio_U_D"] = train["N_Upc"] / train["N_Depts"]
train.shape

(647046, 19)

In [125]:
train["mean_to_min"] = train["Mean"] / train["Min"]
train.loc[train.mean_to_min==0, "mean_to_min"] = 1
train['max_to_mean'] = train["Max"] / train["Mean"]
train.loc[train.max_to_mean==0, "max_to_mean"] = 1

In [126]:
train.drop("Upc", axis=1, inplace=True)
train.shape

(647046, 20)

In [127]:
# 최빈값 함수
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == count]
    return ls[0]

In [128]:
to_grouped = train[["VisitNumber", "FinelineNumber"]]
mf_fineline = to_grouped.groupby("VisitNumber", as_index=False).agg(mode)

In [129]:
depts_dummies = pd.get_dummies(train.DepartmentDescription)
depts_dummies.drop("HEALTH AND BEAUTY AIDS", axis=1, inplace=True)

In [130]:
depts_dummies = pd.concat([train[["VisitNumber"]], depts_dummies], axis=1)
depts_dummies_grouped = depts_dummies.groupby("VisitNumber", as_index=False).sum()
depts_dummies_grouped.shape

(95673, 68)

In [131]:
def smoothing_depts(data):
    tmp = data[["VisitNumber", "Sum"]]
    tmp = tmp.groupby("VisitNumber", as_index=False).max()
    depts_dummies = pd.get_dummies(data["DepartmentDescription"])
    new_df = pd.concat([tmp, depts_dummies], axis=1)
    new_df = new_df.groupby("VisitNumber", as_index=False).sum()
    new_df = new_df.loc[:, "1-HR PHOTO":"WIRELESS"].apply(lambda x: (x+1) / (tmp["Sum"]+68))
    return new_df.drop(["HEALTH AND BEAUTY AIDS"], axis=1)

In [132]:
depts_smoothed = smoothing_depts(train)

In [133]:
train = train.replace(np.inf, 0)
train = train.fillna(value=0)

In [134]:
train.drop(["DepartmentDescription", "FinelineNumber"], axis=1, inplace=True)

In [135]:
train_by_max = train.groupby("VisitNumber", as_index=False).agg(np.max) # ScanCount --> Sum
train_by_sum = train.groupby("VisitNumber", as_index=False).agg(np.sum) 

In [136]:
train_by_sum.head(10)

,VisitNumber,TripType,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,N_Depts,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,5,999,4,-1,-1.0,-1.0,-1.000000,0.0,0.0,0.0,1,1,1.0,-1,1.000000,1.000000,1.000000,1.0
1,7,60,8,2,2.0,2.0,2.000000,0.0,0.0,0.0,4,4,4.0,4,2.000000,2.000000,2.000000,2.0
2,8,598,92,28,23.0,414.0,103.500000,391.0,0.0,0.0,391,483,138.0,644,65.166667,80.500000,103.500000,92.0
3,9,24,12,3,3.0,6.0,4.500000,3.0,0.0,0.0,9,9,6.0,9,4.500000,4.500000,4.500000,4.0
4,10,24,12,3,3.0,6.0,4.500000,3.0,0.0,0.0,9,9,6.0,9,4.500000,4.500000,4.500000,4.0
5,11,140,16,4,4.0,8.0,5.333333,4.0,0.0,0.0,16,16,12.0,16,5.333333,5.333333,5.333333,6.0
6,12,287,28,7,7.0,14.0,12.250000,7.0,0.0,0.0,49,49,28.0,49,12.250000,12.250000,12.250000,8.0
7,15,168,32,9,72.0,72.0,72.000000,0.0,0.0,0.0,48,64,8.0,72,48.000000,64.000000,8.000000,8.0
8,17,24,16,4,8.0,8.0,8.000000,0.0,0.0,0.0,16,16,8.0,16,8.000000,8.000000,4.000000,4.0
9,19,378,36,9,9.0,27.0,13.500000,18.0,0.0,0.0,72,72,54.0,81,12.000000,12.000000,13.500000,18.0


In [137]:
train_grouped = train_by_max.merge(depts_dummies_grouped, how='left', on=["VisitNumber"])
# train_grouped = train_grouped.merge(fineline_dummies_grouped, how='left', on=["VisitNumber"])
train_grouped['ScanCount'] = train_by_sum['ScanCount']
train_grouped.shape

(95673, 85)

In [138]:
depts_tmp = train_grouped.loc[:, "1-HR PHOTO":"WIRELESS"]
depts_tmp = depts_smoothed
depts_tmp.columns = depts_tmp.columns.map(lambda x: str("Prob_") + x)

In [139]:
def grouping_depts(data):
    data["OPTICAL"] = data["OPTICAL - LENSES"] + data["OPTICAL - FRAMES"]
    data["DELI"] = data["SERVICE DELI"] + data["PRE PACKED DELI"]

    data["WOMENS CLOTHES"] = data["PLUS AND MATERNITY"] + data["LADIES SOCKS"] +\
                             data["LADIES SOCKS"] + data["BRAS & SHAPEWEAR"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["PLAYERS AND ELECTRONICS"]
    data["LADIESWEAR"] = data["LADIESWEAR"] + data["WOMENS CLOTHES"]

    data["PHARMACY"] = data["PHARMACY RX"] + data["PHARMACY OTC"]
    data["LAWN AND GARDEN"] = data["LAWN AND GARDEN"] + data["HORTICULTURE AND ACCESS"]
    data["LADIESWEAR"] = data["SHEER HOSIERY"] + data["LADIESWEAR"]

    data["CLOTHES"] = data["BOYS WEAR"] + data["MENSWEAR"] + data["SWIMWEAR/OUTERWEAR"] +\
                      data['GIRLS WEAR, 4-6X  AND 7-14'] + data["INFANT APPAREL"] +\
                      data["LADIESWEAR"] + data["MENS WEAR"] + data["SLEEPWEAR/FOUNDATIONS"]

    data["GROCERIES"] = data["BAKERY"] + data["COMM BREAD"] + data["DAIRY"] + data["DSD GROCERY"] +\
                        data["FROZEN FOODS"] + data["GROCERY DRY GOODS"] + data["MEAT - FRESH & FROZEN"] +\
                        data["PRODUCE"] + data["SEAFOOD"] + data["DELI"]

    data["CLOTHES"] = data["CLOTHES"] + data["SHOES"]
    data["HOME"] = data["BATH AND SHOWER"] + data["HOME DECOR"] + data["HOME MANAGEMENT"] +\
                   data["BEDDING"] + data["COOK AND DINE"] + data["FURNITURE"] +\
                   data["LARGE HOUSEHOLD GOODS"] + data["SEASONAL"]

    data["JEWELRY AND ACCESSORIES"] = data["JEWELRY AND SUNGLASSES"] + data["ACCESSORIES"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["CAMERAS AND SUPPLIES"]

    data['HOUSEHOLD GOODS'] = data['HOUSEHOLD PAPER GOODS'] + data['HOUSEHOLD CHEMICALS/SUPP']
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["WIRELESS"]
    data["BEAUTY AND PERSONAL CARE"] = data["BEAUTY"] + data["PERSONAL CARE"]

    to_drop = ["OPTICAL - FRAMES", "OPTICAL - LENSES", "CONCEPT STORES",
               "OTHER DEPARTMENTS", "PRE PACKED DELI", "SERVICE DELI", "PLUS AND MATERNITY", "LADIES SOCKS",
               "BRAS & SHAPEWEAR", "PLAYERS AND ELECTRONICS", "WOMENS CLOTHES", "PHARMACY RX", "PHARMACY OTC",
               "HORTICULTURE AND ACCESS", "SHEER HOSIERY", "BOYS WEAR", "MENSWEAR", "SWIMWEAR/OUTERWEAR",
               "GIRLS WEAR, 4-6X  AND 7-14", "INFANT APPAREL", "LADIESWEAR", "MENS WEAR", "SLEEPWEAR/FOUNDATIONS",
               "BAKERY", "COMM BREAD", "DAIRY", "DSD GROCERY", "FROZEN FOODS", "GROCERY DRY GOODS",
               "MEAT - FRESH & FROZEN","PRODUCE", "SEAFOOD", "DELI", "SHOES", "HOME DECOR", "BATH AND SHOWER",
               "HOME MANAGEMENT", "BEDDING","COOK AND DINE", "FURNITURE", "LARGE HOUSEHOLD GOODS",
               "SEASONAL", "JEWELRY AND SUNGLASSES", "ACCESSORIES", "CAMERAS AND SUPPLIES",
               "HOUSEHOLD CHEMICALS/SUPP", "HOUSEHOLD PAPER GOODS", "WIRELESS", "BEAUTY", "PERSONAL CARE"]
    
    return data.drop(to_drop, axis=1)

In [140]:
train_depts_group = grouping_depts(train_grouped)
train_depts_group = pd.concat([train_depts_group, depts_tmp], axis=1)
train_depts_group.tail(3)

,VisitNumber,TripType,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,...,Prob_SEAFOOD,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS
95670,191345,39,6,17,1.0,5.0,2.125,4.0,0.0,0.0,...,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765
95671,191346,39,6,17,1.0,5.0,2.125,4.0,0.0,0.0,...,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765
95672,191347,8,6,2,1.0,1.0,1.000,0.0,0.0,0.0,...,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286


In [141]:
# train_depts_group.drop(9999, axis=1, inplace=True)
train_depts_group = pd.concat([train_depts_group, mf_fineline[["FinelineNumber"]]], axis=1)
# train_depts_group = pd.get_dummies(train_depts_group, columns=["Weekday"])
# train_depts_group.drop(X.iloc[:, 170:357], axis=1, inplace=True)

In [429]:
# train_depts_group = pd.get_dummies(train_depts_group, columns=["Weekday"])

In [ ]:
train_depts_group['Return_YN'] = train_depts_group['ScanCount'].apply(lambda x : 1 if x < 0 else 0)
train_depts_group['Return_Count'] = train_depts_group['ScanCount'].apply(lambda x: x if x < 0 else 0)
train_depts_group = train_depts_group.drop(columns = train_depts_group.iloc[:, 44:-2])

In [143]:
print(train_depts_group.shape)
train_depts_group.head()

,VisitNumber,TripType,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,...,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS,FinelineNumber
0,5,999,4,-1,-1.0,-1.0,-1.0,0.0,0.0,0.0,...,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,1000.0
1,7,30,4,2,1.0,1.0,1.0,0.0,0.0,0.0,...,0.014286,0.014286,0.014286,0.028571,0.014286,0.014286,0.014286,0.014286,0.014286,8931.0
2,8,26,4,28,1.0,18.0,4.5,17.0,0.0,0.0,...,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,3565.0
3,9,8,4,3,1.0,2.0,1.5,1.0,0.0,0.0,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,115.0
4,10,8,4,3,1.0,2.0,1.5,1.0,0.0,0.0,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,2008.0


In [144]:
train = train_depts_group

# Test

In [277]:
test = original_test.copy()

In [278]:
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


In [279]:
# 반품 발생은 0으로 표기하는 컬럼 추가
# test["ScanCount"] = test["ScanCount"]
# test.loc[test.ScanCount < 0, "ScanCount"] = 0

# Fineline 없는 것은 9999로 명시
test["FinelineNumber"].fillna(value=9999, inplace=True)

# Upc 없는 것은 -1로 명시
test["Upc"].fillna(value=-1, inplace=True)

In [280]:
wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

test["Weekday"] = test["Weekday"].apply(lambda x: wd[x])

In [281]:
# VisitNumber, DepartmentDescription으로 groupby. ScanCount sum(총 판매량) 
df = test.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].sum()

df1 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].min()
df1.rename(columns={"ScanCount": "Min"}, inplace=True)

df2 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].max()
df2.rename(columns={'ScanCount': "Max"}, inplace=True)

df3 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].mean()
df3.rename(columns={"ScanCount": "Mean"}, inplace=True)

test = test.merge(df1, how='left', on=["VisitNumber"], copy=True)
test = test.merge(df2, how='left', on=["VisitNumber"], copy=True)
test = test.merge(df3, how='left', on=["VisitNumber"], copy=True)  

In [282]:
test["Range"] = test["Max"] - test["Min"]

In [283]:
df = test[test["DepartmentDescription"].isnull()]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Depts"}, inplace=True)
test = test.merge(df, how='left', on=["VisitNumber"], copy=True) 
test["Null_Depts"].fillna(value=1, inplace=True)
test.loc[test.Null_Depts > 0, "Null_Depts"] = 0

In [284]:
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts
0,1,4,7.250339e+10,1,SHOES,3002.0,1.0,2.0,1.333333,1.0,0.0
1,1,4,1.707711e+09,1,DAIRY,1526.0,1.0,2.0,1.333333,1.0,0.0
2,1,4,8.947000e+10,1,DAIRY,1431.0,1.0,2.0,1.333333,1.0,0.0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,1.0,2.0,1.333333,1.0,0.0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,1.0,2.0,1.333333,1.0,0.0


In [285]:
# df = test[test["ScanCount"] < 0]
# df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].ScanCount()
# df.rename(columns={"ScanCount": "ReturnScanCount"}, inplace=True) 
# test = test.merge(df, how='left', on=["VisitNumber"], copy=True)
# test["ReturnCount"].fillna(value=1, inplace=True)
# test.loc[test.ReturnScanCount > 0, "ReturnScanCount"] = 0

In [286]:
# Indicator for missing FinelineNumber

df = test[test["FinelineNumber"]==9999]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Fineline"}, inplace=True)
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)
test["Null_Fineline"].fillna(value=1, inplace=True)
test.loc[test.Null_Fineline > 0, "Null_Fineline"] = 0

In [287]:
# 각 VisitNumber 당 총 구매한 FinelineNumber의 수

df = test.groupby(["VisitNumber",  "FinelineNumber"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Fineline"}, inplace=True) 
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)

In [288]:
# 각 소비자 당 총 구매한 UPC의 수

df = test.groupby(["VisitNumber",  "Upc"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Upc"}, inplace=True) 
test = test.merge(df, how='left', on=["VisitNumber"], copy=True) 

train.head()

,VisitNumber,TripType,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,...,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS,FinelineNumber
0,5,999,4,-1,-1.0,-1.0,-1.0,0.0,0.0,0.0,...,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,1000.0
1,7,30,4,2,1.0,1.0,1.0,0.0,0.0,0.0,...,0.014286,0.014286,0.014286,0.028571,0.014286,0.014286,0.014286,0.014286,0.014286,8931.0
2,8,26,4,28,1.0,18.0,4.5,17.0,0.0,0.0,...,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,3565.0
3,9,8,4,3,1.0,2.0,1.5,1.0,0.0,0.0,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,115.0
4,10,8,4,3,1.0,2.0,1.5,1.0,0.0,0.0,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,2008.0


In [289]:
# 각 소비자 당 총 구매한 DepartmentDescription의 수

df = test.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].count()

In [290]:
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()

In [291]:
df.rename(columns={"ScanCount": "N_Depts"}, inplace=True) 

In [292]:
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)

In [293]:
df = test.groupby(["VisitNumber"], as_index=False)["ScanCount"].sum()
df.rename(columns={"ScanCount": "Sum"}, inplace=True)
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)

In [294]:
test["Ratio_F_D"] = train["N_Fineline"] / train["N_Depts"]
test["Ratio_U_D"] = train["N_Upc"] / train["N_Depts"]
test.shape

(653646, 18)

In [295]:
test["mean_to_min"] = test["Mean"] / test["Min"]
test.loc[test.mean_to_min==0, "mean_to_min"] = 1
test['max_to_mean'] = test["Max"] / test["Mean"]
test.loc[test.max_to_mean==0, "max_to_mean"] = 1

In [296]:
test.drop("Upc", axis=1, inplace=True)
test.shape

(653646, 19)

In [297]:
# 최빈값 함수
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == max_count]
    return ls[0]

In [298]:
to_grouped = test[["VisitNumber", "FinelineNumber"]]
mf_fineline = to_grouped.groupby("VisitNumber", as_index=False).agg(mode)

depts_dummies = pd.get_dummies(test.DepartmentDescription)
# depts_dummies.drop("HEALTH AND BEAUTY AIDS", axis=1, inplace=True)

depts_dummies = pd.concat([test[["VisitNumber"]], depts_dummies], axis=1)
depts_dummies_grouped = depts_dummies.groupby("VisitNumber", as_index=False).sum()
depts_dummies_grouped.shape

(95674, 68)

In [299]:
def smoothing_depts(data):
    tmp = data[["VisitNumber", "Sum"]]
    tmp = tmp.groupby("VisitNumber", as_index=False).max()
    depts_dummies = pd.get_dummies(data["DepartmentDescription"])
    new_df = pd.concat([tmp, depts_dummies], axis=1)
    new_df = new_df.groupby("VisitNumber", as_index=False).sum()
#     new_df = new_df.loc[:, "1-HR PHOTO":"WIRELESS"].apply(lambda x: (x+1) / (tmp["Sum"]+68))
#     return new_df.drop(["HEALTH AND BEAUTY AIDS"], axis=1)
    return new_df

depts_smoothed = smoothing_depts(test)

In [300]:
test = test.replace(np.inf, 0)
test = test.fillna(value=0)

In [301]:
test.head()

,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,N_Depts,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,1,4,1,SHOES,3002.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,1.000000,1.0,1.333333,1.5
1,1,4,1,DAIRY,1526.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,1.000000,1.0,1.333333,1.5
2,1,4,1,DAIRY,1431.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,2.833333,3.5,1.333333,1.5
3,1,4,1,GROCERY DRY GOODS,3555.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,1.500000,1.5,1.333333,1.5
4,2,4,1,DSD GROCERY,4408.0,1.0,2.0,1.333333,1.0,0.0,0.0,3,4,3.0,4,1.500000,1.5,1.333333,1.5


In [302]:
test.drop(["DepartmentDescription", "FinelineNumber"], axis=1, inplace=True)

test_by_max = test.groupby("VisitNumber", as_index=False).agg(np.max) # ScanCount --> Sum
test_by_sum = test.groupby("VisitNumber", as_index=False).agg(np.sum) 

In [303]:
test_grouped = test_by_max.merge(depts_dummies_grouped, how='left', on=["VisitNumber"])
# train_grouped = train_grouped.merge(fineline_dummies_grouped, how='left', on=["VisitNumber"])
test_grouped['ScanCount'] = test_by_sum['ScanCount']
test_grouped.shape

(95674, 84)

In [304]:
test_grouped.head()

,VisitNumber,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1,4,4,1.0,2.0,1.333333,1.0,0.0,0.0,4,...,0,0,0,0,1,0,0,0,0,0
1,2,4,4,1.0,2.0,1.333333,1.0,0.0,0.0,3,...,0,0,0,0,0,0,0,0,0,0
2,3,4,0,0.0,0.0,0.000000,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,4,1,1.0,1.0,1.000000,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,6,4,0,0.0,0.0,0.000000,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0


In [305]:
depts_tmp_test = test_grouped.loc[:, "1-HR PHOTO":"WIRELESS"]
depts_tmp_test = depts_smoothed
depts_tmp_test.columns = depts_tmp_test.columns.map(lambda x: str("Prob_") + x)

In [306]:
def grouping_depts(data):
    data["OPTICAL"] = data["OPTICAL - LENSES"] + data["OPTICAL - FRAMES"]
    data["DELI"] = data["SERVICE DELI"] + data["PRE PACKED DELI"]

    data["WOMENS CLOTHES"] = data["PLUS AND MATERNITY"] + data["LADIES SOCKS"] +\
                             data["LADIES SOCKS"] + data["BRAS & SHAPEWEAR"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["PLAYERS AND ELECTRONICS"]
    data["LADIESWEAR"] = data["LADIESWEAR"] + data["WOMENS CLOTHES"]

    data["PHARMACY"] = data["PHARMACY RX"] + data["PHARMACY OTC"]
    data["LAWN AND GARDEN"] = data["LAWN AND GARDEN"] + data["HORTICULTURE AND ACCESS"]
    data["LADIESWEAR"] = data["SHEER HOSIERY"] + data["LADIESWEAR"]

    data["CLOTHES"] = data["BOYS WEAR"] + data["MENSWEAR"] + data["SWIMWEAR/OUTERWEAR"] +\
                      data['GIRLS WEAR, 4-6X  AND 7-14'] + data["INFANT APPAREL"] +\
                      data["LADIESWEAR"] + data["MENS WEAR"] + data["SLEEPWEAR/FOUNDATIONS"]

    data["GROCERIES"] = data["BAKERY"] + data["COMM BREAD"] + data["DAIRY"] + data["DSD GROCERY"] +\
                        data["FROZEN FOODS"] + data["GROCERY DRY GOODS"] + data["MEAT - FRESH & FROZEN"] +\
                        data["PRODUCE"] + data["SEAFOOD"] + data["DELI"]

    data["CLOTHES"] = data["CLOTHES"] + data["SHOES"]
    data["HOME"] = data["BATH AND SHOWER"] + data["HOME DECOR"] + data["HOME MANAGEMENT"] +\
                   data["BEDDING"] + data["COOK AND DINE"] + data["FURNITURE"] +\
                   data["LARGE HOUSEHOLD GOODS"] + data["SEASONAL"]

    data["JEWELRY AND ACCESSORIES"] = data["JEWELRY AND SUNGLASSES"] + data["ACCESSORIES"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["CAMERAS AND SUPPLIES"]

    data['HOUSEHOLD GOODS'] = data['HOUSEHOLD PAPER GOODS'] + data['HOUSEHOLD CHEMICALS/SUPP']
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["WIRELESS"]
    data["BEAUTY AND PERSONAL CARE"] = data["BEAUTY"] + data["PERSONAL CARE"]

    to_drop = ["OPTICAL - FRAMES", "OPTICAL - LENSES", "CONCEPT STORES",
               "OTHER DEPARTMENTS", "PRE PACKED DELI", "SERVICE DELI", "PLUS AND MATERNITY", "LADIES SOCKS",
               "BRAS & SHAPEWEAR", "PLAYERS AND ELECTRONICS", "WOMENS CLOTHES", "PHARMACY RX", "PHARMACY OTC",
               "HORTICULTURE AND ACCESS", "SHEER HOSIERY", "BOYS WEAR", "MENSWEAR", "SWIMWEAR/OUTERWEAR",
               "GIRLS WEAR, 4-6X  AND 7-14", "INFANT APPAREL", "LADIESWEAR", "MENS WEAR", "SLEEPWEAR/FOUNDATIONS",
               "BAKERY", "COMM BREAD", "DAIRY", "DSD GROCERY", "FROZEN FOODS", "GROCERY DRY GOODS",
               "MEAT - FRESH & FROZEN","PRODUCE", "SEAFOOD", "DELI", "SHOES", "HOME DECOR", "BATH AND SHOWER",
               "HOME MANAGEMENT", "BEDDING","COOK AND DINE", "FURNITURE", "LARGE HOUSEHOLD GOODS",
               "SEASONAL", "JEWELRY AND SUNGLASSES", "ACCESSORIES", "CAMERAS AND SUPPLIES",
               "HOUSEHOLD CHEMICALS/SUPP", "HOUSEHOLD PAPER GOODS", "WIRELESS", "BEAUTY", "PERSONAL CARE"]
    
    return data.drop(to_drop, axis=1)

In [307]:
test_depts_group = grouping_depts(test_grouped)
test_depts_group = pd.concat([test_depts_group, depts_tmp], axis=1)
test_depts_group.tail(3)

,VisitNumber,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,...,Prob_SEAFOOD,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS
95671,191340,6,2,2.0,2.0,2.000000,0.0,0.0,0.0,1,...,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765
95672,191341,6,12,1.0,5.0,2.400000,4.0,0.0,0.0,10,...,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286
95673,191348,6,7,2.0,3.0,2.333333,1.0,0.0,0.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [308]:
# train_depts_group.drop(9999, axis=1, inplace=True)
test_depts_group = pd.concat([test_depts_group, mf_fineline[["FinelineNumber"]]], axis=1)
# train_depts_group = pd.get_dummies(train_depts_group, columns=["Weekday"])
# train_depts_group.drop(X.iloc[:, 170:357], axis=1, inplace=True)

In [ ]:
test_depts_group['Return_YN'] = test_depts_group['ScanCount'].apply(lambda x : 1 if x < 0 else 0)
test_depts_group['Return_Count'] = test_depts_group['ScanCount'].apply(lambda x: x if x < 0 else 0)
test_depts_group = test_depts_group.drop(columns = test_depts_group.iloc[:, 44:-2])

In [309]:
# test_depts_group = pd.get_dummies(test_depts_group, columns=["Weekday"])
test_depts_group.shape

(95674, 112)

In [310]:
test = test_depts_group.copy()

# Data Split

In [147]:
from sklearn.model_selection import train_test_split

In [148]:
# y = train_depts_group["TripType"]
# X = train_depts_group.drop("TripType", axis=1)

In [403]:
X_train, X_test = train_test_split(train, test_size = .4)

In [412]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(X_train.TripType)
X_train_label = le.transform(X_train.TripType) 

In [413]:
le = LabelEncoder()
le.fit(X_test.TripType)
X_test_label = le.transform(X_test.TripType)

# 1) 왜 XGBoost -- predict로 하면 matrix가 나오는지
# 2) DMatrix를 사용하지 않으면 어떤 다른 방법이 있는지.

# XGBoost Modeling

In [173]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import log_loss

In [415]:
X_train_ = X_train.drop(['TripType'], axis=1)
X_test_ = X_test.drop(['TripType'], axis=1)

In [416]:
dtrain = xgb.DMatrix(np.asarray(X_train_), label = np.asarray(X_train_label, dtype=np.uint32))
dtest = xgb.DMatrix(np.asarray(X_test_), label = np.asarray(X_test_label, dtype=np.uint32))

In [417]:
num_round = 300
param1 = {'objective': 'multi:softprob', 'num_class':38, "max_delta_step": 5}
watchlist1 = [(dtrain,'train'), (dtest, 'eval')]

bst = xgb.train(param1, dtrain, num_round, watchlist1, early_stopping_rounds=10)

[0]	train-merror:0.392018	eval-merror:0.405853
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.335819	eval-merror:0.352443
[2]	train-merror:0.325331	eval-merror:0.344813
[3]	train-merror:0.318624	eval-merror:0.339587
[4]	train-merror:0.313642	eval-merror:0.337993
[5]	train-merror:0.309949	eval-merror:0.336687
[6]	train-merror:0.305646	eval-merror:0.334753
[7]	train-merror:0.302563	eval-merror:0.333865
[8]	train-merror:0.297214	eval-merror:0.33167
[9]	train-merror:0.293573	eval-merror:0.330468
[10]	train-merror:0.290386	eval-merror:0.33005
[11]	train-merror:0.28725	eval-merror:0.328377
[12]	train-merror:0.282686	eval-merror:0.326992
[13]	train-merror:0.278557	eval-merror:0.32579
[14]	train-merror:0.276414	eval-merror:0.325007
[15]	train-merror:0.273191	eval-merror:0.325111
[16]	train-merror:0.271083	eval-merror:0.323805
[17]	train-merror:0.26753	eval-merror:0.323465
[18]

In [418]:
X_test_dm = xgb.DMatrix(np.asarray(X_test_))
y_pred = bst.predict(X_test_dm)

In [358]:
y_pred

array([[1.02760496e-04, 2.58847995e-05, 1.36245275e-04, ...,
        1.53898087e-04, 3.90721725e-05, 3.49470824e-02],
       [4.47973543e-05, 1.44515743e-05, 5.05467433e-05, ...,
        4.79401322e-04, 2.07341480e-04, 2.92190835e-02],
       [1.03090257e-04, 1.88102440e-05, 7.41370895e-05, ...,
        1.93656906e-05, 2.93729336e-05, 2.52778418e-02],
       ...,
       [2.09613590e-05, 1.02756327e-04, 2.21976116e-02, ...,
        2.41221916e-02, 1.20933568e-02, 1.96027773e-04],
       [1.86322941e-04, 1.46489574e-05, 5.96270911e-05, ...,
        7.00272387e-03, 6.02135621e-03, 1.16542447e-03],
       [4.70404484e-05, 8.44546139e-06, 3.95534844e-05, ...,
        1.01253181e-05, 1.35754426e-05, 5.93073480e-03]], dtype=float32)

In [419]:
col_array = np.asarray([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37])

In [420]:
columns = {0:"TripType_3", 1: "TripType_4", 2: "TripType_5", 3: "TripType_6", 
                4: "TripType_7", 5: "TripType_8", 6: "TripType_9", 7: "TripType_12", 8: "TripType_14", 
                9: "TripType_15", 10: "TripType_18", 11: "TripType_19", 12: "TripType_20", 13: "TripType_21",
                14: "TripType_22", 15: "TripType_23", 16: "TripType_24", 17: "TripType_25", 18: "TripType_26", 
                19: "TripType_27", 20: "TripType_28", 21: "TripType_29", 22: "TripType_30", 23: "TripType_31", 
                24: "TripType_32", 25: "TripType_33", 26: "TripType_34", 27: "TripType_35", 28: "TripType_36", 
                29: "TripType_37", 30: "TripType_38", 31: "TripType_39", 32: "TripType_40", 33: "TripType_41", 
                34: "TripType_42", 35: "TripType_43", 36: "TripType_44", 37: "TripType_999"}

In [422]:
test_dm = xgb.DMatrix(np.asarray(test))
pred_proba = bst.predict(test_dm)

In [423]:
proba_df = pd.DataFrame(pred_proba, columns=col_array)

In [424]:
proba_df

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,9.825119e-05,7.899294e-06,1.384471e-05,0.000460,1.781234e-01,0.072944,0.028921,2.145575e-05,5.778619e-06,0.020631,...,1.658645e-04,4.551806e-02,1.342250e-01,2.207615e-01,1.919369e-04,6.333007e-03,1.928602e-02,9.567630e-03,8.572307e-04,0.015528
1,1.659194e-04,1.198713e-05,1.686047e-05,0.000642,1.847839e-01,0.019776,0.017465,8.891004e-06,7.904180e-06,0.043081,...,3.249133e-04,6.293025e-02,1.818421e-02,5.245749e-02,5.270436e-05,1.744591e-04,7.480082e-03,2.119966e-03,5.257881e-04,0.065652
2,5.983561e-07,8.467128e-09,1.548789e-07,0.000025,4.061322e-07,0.000080,0.000003,4.542818e-08,1.356185e-08,0.000001,...,8.054865e-04,7.933498e-08,1.106160e-07,8.273710e-07,7.354941e-08,3.936538e-08,5.820982e-07,2.522616e-08,8.388403e-07,0.999077
3,6.924721e-05,9.881994e-07,7.467627e-05,0.000624,4.559156e-05,0.026715,0.913868,4.163404e-06,6.670919e-05,0.000684,...,5.061434e-05,6.532801e-05,1.737740e-05,7.755431e-05,1.205387e-05,1.611440e-05,5.805158e-04,8.096294e-06,2.900163e-05,0.051175
4,7.968398e-06,1.825517e-07,9.190153e-07,0.000002,1.113491e-06,0.000079,0.000075,1.264687e-07,7.394280e-08,0.000016,...,8.726666e-07,3.808205e-07,4.151320e-07,2.619107e-06,4.001815e-07,3.340083e-06,8.529828e-06,4.957390e-07,1.283130e-06,0.999620
5,8.324001e-05,1.002018e-06,1.692065e-05,0.000552,4.167190e-01,0.564069,0.005357,2.218292e-06,1.117932e-06,0.000325,...,2.930136e-05,4.607425e-03,5.681326e-04,2.271690e-04,9.882351e-06,4.235076e-06,8.245481e-05,5.418853e-06,1.003021e-05,0.006365
6,3.077736e-05,7.945137e-06,5.846062e-05,0.000070,5.773913e-02,0.000315,0.000171,4.697944e-05,1.736334e-05,0.185607,...,3.554343e-04,3.100434e-02,3.201651e-02,3.704820e-01,9.695812e-03,6.196289e-04,5.361454e-02,1.287817e-01,2.108739e-02,0.001424
7,1.933353e-05,4.307046e-06,3.668069e-05,0.000034,3.390818e-02,0.000048,0.000028,5.711620e-06,1.453403e-04,0.118439,...,1.991047e-04,8.235808e-02,2.747984e-02,1.941407e-01,2.024155e-02,7.545780e-05,3.757414e-04,4.455128e-03,9.567626e-04,0.006692
8,3.528897e-05,5.623198e-06,1.078767e-04,0.000195,2.871492e-01,0.000097,0.000141,3.260097e-05,9.223173e-06,0.043368,...,4.052204e-04,8.474022e-02,9.630165e-02,3.246743e-01,4.060959e-04,1.437739e-04,3.962682e-04,1.231650e-03,3.999376e-04,0.003662
9,1.118166e-04,4.112335e-03,3.957510e-01,0.000241,1.154831e-03,0.000403,0.001034,1.118025e-05,7.679932e-06,0.009793,...,5.545569e-03,1.905097e-02,9.254680e-03,3.933230e-01,2.948044e-04,3.395978e-03,7.705927e-03,3.372322e-02,4.340963e-03,0.011376


In [425]:
proba_df.columns = proba_df.columns.map(lambda x: columns[x])

In [426]:
sub_df = pd.concat([test[["VisitNumber"]], proba_df], axis=1)

In [427]:
sub_df

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,9.825119e-05,7.899294e-06,1.384471e-05,0.000460,1.781234e-01,0.072944,0.028921,2.145575e-05,5.778619e-06,...,1.658645e-04,4.551806e-02,1.342250e-01,2.207615e-01,1.919369e-04,6.333007e-03,1.928602e-02,9.567630e-03,8.572307e-04,0.015528
1,2,1.659194e-04,1.198713e-05,1.686047e-05,0.000642,1.847839e-01,0.019776,0.017465,8.891004e-06,7.904180e-06,...,3.249133e-04,6.293025e-02,1.818421e-02,5.245749e-02,5.270436e-05,1.744591e-04,7.480082e-03,2.119966e-03,5.257881e-04,0.065652
2,3,5.983561e-07,8.467128e-09,1.548789e-07,0.000025,4.061322e-07,0.000080,0.000003,4.542818e-08,1.356185e-08,...,8.054865e-04,7.933498e-08,1.106160e-07,8.273710e-07,7.354941e-08,3.936538e-08,5.820982e-07,2.522616e-08,8.388403e-07,0.999077
3,4,6.924721e-05,9.881994e-07,7.467627e-05,0.000624,4.559156e-05,0.026715,0.913868,4.163404e-06,6.670919e-05,...,5.061434e-05,6.532801e-05,1.737740e-05,7.755431e-05,1.205387e-05,1.611440e-05,5.805158e-04,8.096294e-06,2.900163e-05,0.051175
4,6,7.968398e-06,1.825517e-07,9.190153e-07,0.000002,1.113491e-06,0.000079,0.000075,1.264687e-07,7.394280e-08,...,8.726666e-07,3.808205e-07,4.151320e-07,2.619107e-06,4.001815e-07,3.340083e-06,8.529828e-06,4.957390e-07,1.283130e-06,0.999620
5,13,8.324001e-05,1.002018e-06,1.692065e-05,0.000552,4.167190e-01,0.564069,0.005357,2.218292e-06,1.117932e-06,...,2.930136e-05,4.607425e-03,5.681326e-04,2.271690e-04,9.882351e-06,4.235076e-06,8.245481e-05,5.418853e-06,1.003021e-05,0.006365
6,14,3.077736e-05,7.945137e-06,5.846062e-05,0.000070,5.773913e-02,0.000315,0.000171,4.697944e-05,1.736334e-05,...,3.554343e-04,3.100434e-02,3.201651e-02,3.704820e-01,9.695812e-03,6.196289e-04,5.361454e-02,1.287817e-01,2.108739e-02,0.001424
7,16,1.933353e-05,4.307046e-06,3.668069e-05,0.000034,3.390818e-02,0.000048,0.000028,5.711620e-06,1.453403e-04,...,1.991047e-04,8.235808e-02,2.747984e-02,1.941407e-01,2.024155e-02,7.545780e-05,3.757414e-04,4.455128e-03,9.567626e-04,0.006692
8,18,3.528897e-05,5.623198e-06,1.078767e-04,0.000195,2.871492e-01,0.000097,0.000141,3.260097e-05,9.223173e-06,...,4.052204e-04,8.474022e-02,9.630165e-02,3.246743e-01,4.060959e-04,1.437739e-04,3.962682e-04,1.231650e-03,3.999376e-04,0.003662
9,21,1.118166e-04,4.112335e-03,3.957510e-01,0.000241,1.154831e-03,0.000403,0.001034,1.118025e-05,7.679932e-06,...,5.545569e-03,1.905097e-02,9.254680e-03,3.933230e-01,2.948044e-04,3.395978e-03,7.705927e-03,3.372322e-02,4.340963e-03,0.011376


In [428]:
sub_df.to_csv("submission_xgb_2.csv", index=False)

그 외 적용점 

In [330]:
np.array(X_test.TripType)

array([ 8, 36,  8, ..., 39, 39,  8])

In [240]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import LabelBinarizer

In [245]:
lb = LabelBinarizer()
lb.fit(X_test_label)
y2 = lb.transform(X_test_label)

In [250]:
y2

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
print(classification_report(y2, y_pred))

In [ ]:
dtrain = xgb.DMatrix(data=X, label=np.array([range(len(X))], dtype=np.uint32).T)

def objFunc(..., dtrain):
    indexes=dtrain.get_label().astype('uint32')
    dtrain=X.iloc[indexes]
    return g, h

bst = xgb.train(..., obj=objFunc, ...)

In [331]:
walmart_test = xgb.DMatrix(np.asarray(test))
prediction_xgb = bst.predict(walmart_test)

In [334]:
prediction_xgb.astype(int)
prediction_xgb

array([[2.2835880e-05, 1.0106867e-05, 2.5292809e-05, ..., 2.1066862e-04,
        6.2925195e-05, 8.6761367e-01],
       [5.8068428e-04, 4.2971147e-05, 2.2810197e-04, ..., 3.6148480e-04,
        1.8670391e-04, 1.6760890e-01],
       [3.0411238e-05, 5.4108241e-06, 4.9307309e-05, ..., 3.6149766e-05,
        3.0830887e-04, 2.8928006e-01],
       ...,
       [4.6393918e-03, 1.0913062e-04, 7.6271594e-04, ..., 2.3896897e-03,
        2.1824802e-03, 5.2670218e-02],
       [3.1299838e-05, 5.4110242e-06, 7.9155443e-06, ..., 6.9684570e-04,
        8.3621610e-03, 4.6911049e-03],
       [1.9105099e-05, 7.5711046e-06, 9.7501761e-05, ..., 1.5369616e-03,
        3.1739142e-02, 2.6002913e-03]], dtype=float32)

In [ ]:
def prediction_to_csv(prediction):
    df_VN = pd.DataFrame(columns=["VisitNumber"])
    df_VN["VisitNumber"] = test.index
    df_TT = pd.DataFrame(columns=["TripType"])
    df_TT["TripType"] = prediction
    df_TT = pd.get_dummies(df_TT.TripType).astype(int, cbopy=False)
    df_TT = df_TT.rename(columns = {0:"TripType_3", 1: "TripType_4", 2: "TripType_5", 3: "TripType_6", 
                4: "TripType_7", 5: "TripType_8", 6: "TripType_9", 7: "TripType_12", 8: "TripType_14", 
                9: "TripType_15", 10: "TripType_18", 11: "TripType_19", 12: "TripType_20", 13: "TripType_21",
                14: "TripType_22", 15: "TripType_23", 16: "TripType_24", 17: "TripType_25", 18: "TripType_26", 
                19: "TripType_27", 20: "TripType_28", 21: "TripType_29", 22: "TripType_30", 23: "TripType_31", 
                24: "TripType_32", 25: "TripType_33", 26: "TripType_34", 27: "TripType_35", 28: "TripType_36", 
                29: "TripType_37", 30: "TripType_38", 31: "TripType_39", 32: "TripType_40", 33: "TripType_41", 
                34: "TripType_42", 35: "TripType_43", 36: "TripType_44", 37: "TripType_999"})
    trial = pd.concat([df_VN, df_TT], axis=1).set_index('VisitNumber')
    #trial["TripType_4"] = 0
    trial["TripType_14"] = 0
    return trial.to_csv("data/trial_6.csv")